In [1005]:
import pandas as pd
import json
import numpy as np

In [1006]:
dg=pd.read_excel('data/clean/erdely3_manual.xlsx').drop('Unnamed: 0',axis=1)

Clean up names

In [1007]:
print(len(dg))
dg=dg.set_index(['Év','Megye','Település','Név','Típus','Profil']).dropna(how='all').reset_index()
dg['ID']=dg.index
print(len(dg))

22007
21523


In [1008]:
megye={'Fehér':'AB', 'Arad':'AR', 'Bukarest':'B', 'Bákó':'BC', 'Bihar':'BH', 'Beszterce-Naszód':'BN',
       'Brassó':'BV', 'Kolozs':'CJ', 'Kovászna':'CV', 'Krassó-Szörény':'CS', 'Hunyad':'HD',
       'Hargita':'HR', 'Máramaros':'MM', 'Maros':'MS', 'Szeben':'SB', 'Szatmár':'SM', 'Szilágy':'SJ',
       'Temes':'TM'}

In [1009]:
from difflib import SequenceMatcher
from difflib import get_close_matches

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [1010]:
def replacer(s,d):
    for i in d:
        s=s.replace(i,d[i])
    return s

In [1011]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

Import data.gov.ro etalon data

In [1012]:
ro={'Ă':'A','Ş':'S','Â':'A','Ș':'S','Ț':'T','Â':'A','Î':'I','Ă':'A','Ţ':'T','-':' ','SC.GEN.':'','I VIII':''}
def roman(s):
    return replacer(s,ro)

In [1013]:
gov=pd.read_excel('data/raw/data.gov.ro/datedeschise-retea-2019-2020.xls')

In [1014]:
locs=gov.set_index('Judet')['Localitate']

In [1035]:
m='Fehér'

In [1036]:
d1=dg[dg['Megye']==m][['Település','Név','ID']]
d1['telepules']=[roman(telepules) for telepules in d1['Település'].astype('str').str.upper().str.strip()]
locs1=locs.loc[megye[m]].unique()
locs2=np.sort([roman(i) for i in locs1])

In [1037]:
no_locs={
    'Fehér':['GAMBAS','STANA DE MURES'],
    'Arad':['FINIS','SUPLACU DE BARCAU'], 
    'Bukarest':[], 
    'Bákó':['GHIMES-FAGET'], 
    'Bihar':['POSOLOACA','GIRISU DE CRIS','DOLEA'], 
    'Beszterce-Naszód':['FANTANITA','STRUGURENI'],
    'Brassó':[], 
    'Kolozs':['INUCU'], 
    'Kovászna':['VALEA SCURTA','DOBOLII DE JOS','DOBOLII DE SUS','LUTOASA','TOFALAU','ZOLTAN'], 
    'Krassó-Szörény':[], 
    'Hunyad':[],
    'Hargita':['ALEXANDRITA','CINOD','DEJUTIU','DIAFALAU','FILIASI','GHIPES','PETECU','RARES','SICASAU','SICLOD','BADENI'], 
    'Máramaros':[], 
    'Maros':['CUCERDEA','DAMIENI','ISTICEU','PAUCISOARA','SANTANDREI'], 
    'Szeben':[], 
    'Szatmár':[], 
    'Szilágy':[],
    'Temes':[]
}

init_override={
    'Fehér':{
        'ALBA':'ALBA IULIA',
        'IULIA':'ALBA IULIA',
        'GRANOASA':'BUCERDEA GRANOASA',
        'MURES':'STANA DE MURES',
        'ST. DE MURES':'STANA DE MURES',
        'NAN':'OIEJDEA',
        'NOUA':'LOPADEA NOUA',
        'AIUDUL DE SUS':'AIUD',
        'BALTA':'CETATEA DE BALTA',
        'STINA DE MURES':'STANA DE MURES',
        '“T.COCISIU”BLAJ':'BLAJ'
    },
    'Arad':{
        'CRIS':'CHISINEU-CRIS',
        'NEAGRA':'IERMATA NEAGRA',
        'ZIMAND CUZ':'ZIMANDCUZ'
    },
    'Bukarest':{
        'BUCURESTI':'BUCURESTI SECTORUL 2',
        'ENDRE':'BUCURESTI SECTORUL 2'
    },
    'Bákó':{
        'GERGELY”':'GHIMES',
        'GHIMES FAGET':'GHIMES-FAGET',
        'GHIMES FEGET':'GHIMES-FAGET'
    },
    'Bihar':{
        'BARCAU':'SUPLACU DE BARCAU',
        'BRANCUSI"ORADEA':'ORADEA',
        'EMINESCU"':'ORADEA',
        'G.S.A.E.SALARD':'SALARD',
        'GOGA,,':'MARGHITA',
        'JOS':'LUGASU DE JOS',
        'MIHAI':'VALEA LUI MIHAI',
        'SCHILLER"ORADEA':'ORADEA',
        'SIMIAN (SIMIAN NOU   BARANTAU)':'SIMIAN NOU',
        'SIMIAN NOU  BARANTAU':'SIMIAN NOU',
        'SLAVICI"ORADEA':'ORADEA',
        'VITEAZUL"':'ORADEA',
        'CRIS':'GIRISU DE CRIS',
        'GHIBU"':'ORADEA',
        'VULCAN"':'ORADEA',
        'NAN':'CADEA'
    },
    'Beszterce-Naszód':{
        'FIINTINELE ST':'FANTANELE',
        'FINTINITA':'FANTANITA',
        'NIMIGEA':'NIMIGEA DE JOS',
        'TEOR.TEACA':'TEACA',
        "REBREANU’'B TA":'BISTRITA',
        'BRETEA':'SIEU ODORHEI'
    },
    'Brassó':{
        'VECHI':'TICUSU VECHI'
    },
    'Kolozs':{
        'BICALAT':'HUDEIN',
        'BICALATU':'HUDEIN',
        'HUEDIN BICALAT':'HUDEIN',
        'CORNESTI (COMUNA MIHAI VITEAZU)':'MIHAI VITEAZU',
        'MIC':'CAPUSU MIC',
        'INUC':'INUCU',
        'RASCRUCI, JUD. CLUJ':'RASCRUCI',
        'SUS':'LUNA DE SUS',
        'VALENI, COM. CALATELE':'CALATELE',
        'TRITENI COLONIA':'COLONIA',
        'TRITENI COLONIE':'COLONIA',
        'TRITENI COLONNIE':'COLONIA'
    },
    'Kovászna':{
        'BAI':'MALNAS BAI',
        'CALEA SCURTA':'VALEA SCURTA',
        'COMOLAU':'RECI',
        'COMOLAU   RECI':'RECI',
        'GHEORGHE':'SFANTU GHEORGHE',
        'LASZLO"ILIENI':'ILIENI',
        'MAJNAS SAT':'MALNAS',
        'SEACA':'VALEA SEACA',
        'SCURTA':'VALEA SCURTA',
        'MALNAS SAT':'MALNAS',
        'LUNCA OZUNULUI':'OZUN',
        'TINOASA':'TARGU SECUIESC'
    },
    'Hargita':{
        'ANTALOC':'LUNCA DE JOS',
        'CATUN BECAS':'BECAS',
        'FRUMOASA SAT. BARZAVA':'BARZAVA',
        'DITRAU/JOLOTCA':'DITRAU',
        'M CIUC':'MIERCUREA CIUC',
        'JIGODIN BAI MANO TANODA':'JIGODIN BAI',
        'SUBPADURE':'JIGODIN BAI',
        'FRUMOASA SAT. NICOLESTI':'NICOLESTI',
        'JOLOTCA/TILALMAS':'JOLOTCA',
        'JOSENI   BORZONT':'BORZONT',
        'ODORHEIU':'ODORHEIU SECUIESC',
        'LUNCA DE JOS, VALEA RECE':'LUNCA DE JOS',
        'VALEA CAPELEI':'LUNCA DE JOS',
        'M CIUC STR TOPLITA':'MIERCUREA CIUC',
        'OCNA DE SUSSCOALA GIMNAZIALA "SZEKELY MOZES"':'OCNA DE SUS',
        'VOSLABENI HEVEDER':'VOSLABENI',
        'SCOALA GIMN."TORO TIBOR" ULIES':'ULIES',
        'SUMULEU':'MIERCUREA CIUC',
        'SCOALA PRIMARA DAIA':'DAIA',
        'SCOALA PRIMARA PETECU':'PETECU',
        'VALEA LUI ANTALOC':'LUNCA DE JOS',
        'VARSAG CENTRU':'VARSAG',
        'VARSAG CENTU':'VARSAG',
        'VARSAG FANTINA MARE':'VARSAG',
        'VARSAG LAZURELE':'VARSAG',
        'VARSAG PIATRA SOIMULUI':'VARSAG',
        'SUSENI MURES SAT':'SUSENI',
        'BETESTI':'CRISTURU SECUIESC',
        'OTENI':'TAURENI'
    },
    'Máramaros':{
        'MANAU':'ULMENI',
        'VULCAN':'BAIA MARE',
        'TISA':'CAMPULUNG LA TISA',
        'MINAU':'ULMENI',
        'SIGHETU':'SIGHETU MARMATIEI'
    },
    'Maros':{
        'BISTRA   VALE':'BISTRA MURESULUI',
        'BOBOHALMA   TARNAVENI':'TARNAVENI',
        'BRANCOV':'BRANCOVENESTI',
        'CACICIU +NADASA':'NADASA',
        'CEUASU DE CIMPIE':'CEUASU DE CAMPIE',
        'CHIHERU':'CHIHERU DE JOS',
        'MIERCUREA':'MIERCUREA NIRAJULUI',
        'CIBU':'FANTANELE',
        'COSBUC"TG.MURES':'TARGU MURES',
        'CIOBOTANI':'STANCENI',
        'COMORI':'GURGHIU',
        'CORNESTI  ADAMUS':'ADAMUS',
        'CRAIESTI  ADAMUS':'ADAMUS',
        'GH.DOJA':'GHEORGHE DOJA',
        'GH. DOJA':'GHEORGHE DOJA',
        'D VIOAR':'DUMBRAVIOARA',
        'D VIOARA':'DUMBRAVIOARA',
        'DAMBAU  ADAMUS':'ADAMUS',
        'DUBISTE':'IBANESTI',
        'ERCEA':'BALA',
        'EREMIENI+DROJDII':'EREMIENI',
        'GRUISOR SUVEICA':'GRUISOR',
        'LUDUS   ROSIORI':'LUDUS',
        'M.NIRAJ':'MIERCUREA NIRAJULUI',
        'MAIA +CANDU':'MAIAD',
        'MOISA+MERISOR':'MOISA',
        'POARTA':'FARAGAU',
        'PUSTA':'SINCAI',
        'RAPA  DE JOS':'VATAVA',
        'SANG. DE M':'SANGEORGIU DE MURES',
        'SG. DE MS.':'SANGEORGIU DE MURES',
        'SANPETRU DE CAMPIE (SANGEORGIU DE CAMPIE)':'SANGEORGIU DE CAMPIE',
        'SEUCA':'GANESTI',
        'SG P':'SANGEORGIU DE PADURE',
        'SG.DE P':'SANGEORGIU DE PADURE',
        'SIGHI':'SIGHISOARA',
        'T VENI':'TARNAVENI',
        'TG M':'TARGU MURES',
        'TG.M':'TARGU MURES',
        'SUVEICA':'GRUISOR',
        'TOPA':'ALBESTI',
        'TUSIN':'SANPETRU DE CAMPIE',
        'VALEA GLODULUI':'VALEA LARGA',
        'DILEU NOU':'SANPAUL',
        'SACADATU DE JOS':'SOVATA',
        'ADRIANU MARE':'ADRIAN',
        'BOZED':'BEZID',
        'DEAG':'IERNUT',
        'DUMBRAV.':'DUMBRAVIOARA',
        'MIC':'BUDIU MIC'
    },
    'Szeben':{
        'COPSA':'COPSA MARE',
        'ALMA   SMIG':'SMIG'
    },
    'Szatmár':{
        'MARE':'SATU MARE',
        'NADIS':'NADISU HODODULUI',
        'OASULUI':'ORASU NOU',
        'PELISOR':'LAZURI',
        'NOU':'ORASU NOU'
    },
    'Szilágy':{
        'AGHIRES/AGHIRES':'AGHIRES',
        'ARCHID COSEIU':'ARCHID',
        'BIUSA ALUNIS':'BIUSA',
        'CRISTUR':'CRISTUR CRISENI',
        'DIOSOD PANIC':'DIOSOD',
        'GIRCEIU/GIRCEIU':'GIRCEIU',
        'UILEACU SIMLEULUI':'MAERISTE'
    }
}

def init(s,m):
    if m in init_override:
        if s in init_override[m]:
            return init_override[m][s]
    return s

In [1040]:
for telepules in np.sort(d1['telepules'].unique()):
    t=init(telepules,m)
    s=process.extractOne(t, locs2, scorer=fuzz.token_sort_ratio)
    if s[1]>70: s=s[0]
    else: s=''
    if t in no_locs[m]: s=t
#     print(telepules,'['+s+']')
    if ((s=='') or (t=='NAN') or (t=='')):
        print(telepules,'['+s+']')
        print(d1[d1['telepules']==telepules]['Név'])


Assign new telepules names

In [1026]:
mlocs={}
for m in megye:
    locs1=locs.loc[megye[m]].unique()
    mlocs[m]=np.sort([roman(i) for i in locs1])

In [1054]:
mtelepules={}
for m in megye:
    print(m)
    locs1=locs.loc[megye[m]].unique()
    locs2=np.sort([roman(i) for i in locs1])
    mlocs[m]=np.sort([roman(i) for i in locs1])
    d1=dg[dg['Megye']==m][['Település','Név','ID']]
    for telepules in np.sort(d1['Település'].astype(str).unique()):
        t=roman(telepules.upper().strip())
        t=init(t,m)
        s=process.extractOne(t, locs2, scorer=fuzz.token_sort_ratio)
        if s[1]>70: s=s[0]
        else: s=''
        if t in no_locs[m]: s=t
        # print(telepules,'['+s+']')
        if ((s=='') or (t=='NAN') or (t=='')):
            print(telepules,'['+s+']')
            print(d1[d1['telepules']==telepules]['Név'])
        if m not in mtelepules:
            mtelepules[m]={}
        mtelepules[m][telepules]=s

Fehér
Arad
Bukarest
Bákó
Bihar
Beszterce-Naszód
Brassó
Kolozs
Kovászna
Krassó-Szörény
Hunyad
Hargita
Máramaros
Maros
Szeben
Szatmár
Szilágy
Temes


In [1056]:
telepules_norm=[]
for i in list(dg.T.iteritems()):
    m=i[1][1]
    locs2=mlocs[m]
    telepules=roman(str(i[1][2]).upper().strip())
    t=init(telepules,m)
    s=process.extractOne(t, locs2, scorer=fuzz.token_sort_ratio)
    if s[1]>70: s=s[0]
    else: s=''
    if t in no_locs[m]: s=t
    telepules_norm.append(s)
dg['telepules']=telepules_norm

In [1058]:
dg.to_excel('data/clean/erdely4.xlsx')

Resolve school names

In [1324]:
dg=pd.read_excel('data/clean/erdely4_manual.xlsx').drop('Unnamed: 0',axis=1)

Edit manually, remove duplicates / totals

In [1325]:
totals=[12479,12484,12488,12494,12499,12509,12517,12521,12529,12534,12541,12545,12548,12551,12557,12565,12570,12574,12581,12584,
       12586,12590,12598,12603,12606,12608,12613,12616,12619,12628,12634,12640,12650,12653,12658,12665,12668,18728]
dg=dg[~dg['ID'].isin(totals)]

Consistency testing

- Osztalyok szama

In [1327]:
#10,12,14,16,18,19,20,21,23,25,27,29,31,33,35,37,39,41,43,45,47,49,51,53
c=dg.columns[55]
print(c)

14. osztályok száma


In [1328]:
dk=dg.set_index(['Év', 'Megye', 'Település','ID'])[c].astype(str).str.replace(',','.').str.replace('-','nan')\
    .str.replace('*','nan').str.replace('16nan','16').replace('nan0nan','nan').str.replace(' ','nan').str.replace('nannannan','nan')\
    .str.replace('nannan','nan').str.replace('?','nan').str.replace('0.nan25','nan').str.replace('nan0.5','nan').str.replace('nannan','nan').astype(float)
dk[dk>5]

Series([], Name: 14. osztályok száma, dtype: float64)

In [1329]:
for c in ['Óvodás csoportok összesen', 'Óvodások összesen',
       'Kiscsoportok száma', 'Kiscsoportosok', 'Középcsoportok száma',
       'Középcsoportosok', 'Nagycsoportok száma', 'Nagycsoportosok',
       'Vegyes csoportok száma', 'Vegyescsoportosok', 'Tanítók összesen',
       'Képzett tanítók', 'Képzetlen tanítók', 'Elemi osztályok összesen',
       'Elemisek összesen', 'Előkészítő osztályok száma',
       'Előkészítő osztályosok', '1. osztályok száma', '1. osztályosok',
       '2. osztályok száma', '2. osztályosok', '3. osztályok száma',
       '3. osztályosok', '4. osztályok száma', '4. osztályosok',
       'Általános osztályok összesen', 'Általánososok összesen',
       '5. osztályok száma', '5. osztályosok', '6. osztályok száma',
       '6. osztályosok', '7. osztályok száma', '7. osztályosok',
       '8. osztályok száma', '8. osztályosok',
       'Középiskolai osztályok összesen', 'Középiskolások összesen',
       '9. osztályok száma', '9. osztályosok', '10. osztályok száma',
       '10. osztályosok', '11. osztályok száma', '11. osztályosok',
       '12. osztályok száma', '12. osztályosok', '13. osztályok száma',
       '13. osztályosok', '14. osztályok száma', '14. osztályosok']:
    print(c)
    dg[c]=dg[c].astype(str).str.replace(',','.').str.replace('-','nan')\
    .str.replace('*','nan').str.replace('16nan','16').replace('nan0nan','nan').str.replace(' ','nan').str.replace('nannannan','nan')\
    .str.replace('nannan','nan').str.replace('?','nan').str.replace('0.nan25','nan').str.replace('nan0.5','nan')\
    .str.replace('nannan','nan').str.replace('`','').str.replace('120nan','120').astype(float)

Óvodás csoportok összesen
Óvodások összesen
Kiscsoportok száma
Kiscsoportosok
Középcsoportok száma
Középcsoportosok
Nagycsoportok száma
Nagycsoportosok
Vegyes csoportok száma
Vegyescsoportosok
Tanítók összesen
Képzett tanítók
Képzetlen tanítók
Elemi osztályok összesen
Elemisek összesen
Előkészítő osztályok száma
Előkészítő osztályosok
1. osztályok száma
1. osztályosok
2. osztályok száma
2. osztályosok
3. osztályok száma
3. osztályosok
4. osztályok száma
4. osztályosok
Általános osztályok összesen
Általánososok összesen
5. osztályok száma
5. osztályosok
6. osztályok száma
6. osztályosok
7. osztályok száma
7. osztályosok
8. osztályok száma
8. osztályosok
Középiskolai osztályok összesen
Középiskolások összesen
9. osztályok száma
9. osztályosok
10. osztályok száma
10. osztályosok
11. osztályok száma
11. osztályosok
12. osztályok száma
12. osztályosok
13. osztályok száma
13. osztályosok
14. osztályok száma
14. osztályosok


In [1331]:
dg.to_excel('data/clean/erdely5.xlsx')

- Diak/osztaly ratio

In [2998]:
import re

In [3559]:
dg=pd.read_excel('data/clean/erdely5_manual.xlsx').drop('Unnamed: 0',axis=1)

In [3561]:
#10,12,14,16,18,19,20,21,23,25,27,29,31,33,35,37,39,41,43,45,47,49,51,53,55
i=55
c=dg.columns[i]
c1=dg.columns[i+1]
print(c,c1)

14. osztályok száma 14. osztályosok


In [3562]:
dk=dg.set_index(['Év', 'Megye', 'Település','ID'])
dk=dk[dk[c]>0]
dk['ratio']=dk[c1]/dk[c]
dk=dk['ratio'].dropna()
dk[dk>50]

Series([], Name: ratio, dtype: float64)

Sort out Nev/Tipus/Profil misalignments

PJ, ST replace with loc name  
ST = STRUCTURA  
PJ = PERSONALITATE JURIDICA  
UT = UNITATE TERITORIALA  

In [4126]:
mgov_clean={}

In [4312]:
m=list(megye.keys())[17]
print(m)
# for i,d in enumerate(np.sort(dg[dg['Megye']==m]['Név'].unique())):
#     print(i,d)

Temes


Automatic recognition of school names: 
- find if telepules substring in school
- if not, append to school name end
- then do matching

In [4313]:
ro={'Ă':'A','Ş':'S','Â':'A','Ș':'S','Ț':'T','Â':'A','Î':'I','Ă':'A','Ţ':'T',
   'Á':'A','É':'E','Í':'I','Ó':'O','Ő':'O','Ö':'O','Ú':'U','Ű':'U','Ü':'U'}
def roman(s):
    return replacer(s,ro)

In [4314]:
def cleaner(nev,telepules,convert_ro=True,test=False):
    for i in range(50,1,-1):
        nev=nev.replace(i*' ',' ')
    if nev in full_replace:
        return full_replace[nev]+' '+telepules
    else:
        for s in to_clean_before:
            for i in to_clean_before[s]:
                if i in nev:
                    nev=nev.replace(i,' '+s+' ')
        for s in to_clean:
            for i in to_clean[s]:
                if i in nev:
                    nev=nev.replace(i,' '+s+' ')
                if convert_ro:
                    if roman(i) in nev:
                        nev=nev.replace(roman(i),' '+s+' ')
        for s in to_clean2:
            for i in to_clean2[s]:
                if i in nev:
                    nev=nev.replace(i,' '+s+' ')
                if convert_ro:
                    if roman(i) in nev:
                        nev=nev.replace(roman(i),' '+s+' ')
        for s in to_clean_after:
            for i in to_clean_after[s]:
                if i in nev:
                    nev=nev.replace(i,' '+s+' ')
    for i in range(50,1,-1):
        nev=nev.replace(i*' ',' ')
    include_fr=False
#     fr=fuzz.ratio(telepules,nev)
#     if len(telepules)<6:
#         if fr<15: include_fr=True
#     else:
#         if fr<25: include_fr=True
    tnev=replacer(nev,  {i:'' for i in list(to_clean.keys())[:to_clean_k]+['#']}).strip()
    fr=fuzz.partial_ratio(telepules,tnev)
    if fr<70: include_fr=True
    if include_fr:
        nev=nev+' '+telepules
    for s in list(to_clean.keys())[:to_clean_k]:
        if nev.count(s)>1:
            nev=re.sub(s[::-1],"",nev[::-1],count=1)[::-1]
    if not test:
        nev=nev.replace('#','').replace('#','').replace('#','')
    for i in range(10,1,-1):
        nev=nev.replace(i*' ',' ')
    return nev.strip()

In [4315]:
to_clean_before={
    'GRDPLG BABSZEM JANKO':['GRĂDINIȚA ”BABSZEM JANKO” CU PROGR PREL.'],
    'COLGNAT BETHLEN GABOR':['COLEGIUL NATIONAL”BG”'],
}
to_clean={
    'SGMZ':['SCOALA GIMNAZIALĂ','ŞCOALA CU CLASELE I-VIII','ȘCOALA CU CLASELE I-VIII','ŞCOALA GIMNAZIALĂ','SOALA GIMN.',' SC.GIMN.','SCOALA GIMNAZIALA',
           'ȘCOALA GIMNAZIALĂ','SC. G.','SC. GIMN.','SCOALA GIM.','SO8',
           'ŞC GIMNAZIALA','ŞC GIMNAZIALA','ŞCOALA CU CLS. I-VIII','ŢCOALA GIMNAZIALĂ','ȘC.GIM.','ȘCOALA GIMNAZIALA',
           'S08','SC.GIMN.','SCOALA CU CLASELE I-VIII','SCOALA CU CLS.I-VIII','SC.GIMNAZIALA','SCOALA GINAZIALA',
           'SCOALA CU CLASELE I - VIII','ŞCOALA CU CLS I-VIII','ŞC.CU CLS. I-VIII','ŞCOALA GIMNAYIALĂ','ŞC.CU CLS.I-VIII','Ş08',
           'ŞC. CU CLS. I-VIII','SCOALA CU CLS I-VIII','ŞCOALA CU CLS.I-VIII','ŞCOALA GIMNAYIALA', 'ŞCOLALA GIMNAZIALĂ',
           'ŞCOALA GIMNAZIALA','ŞCOALA GIMNAZILĂ','ŞCOALA CU CL.I-VIII','ȘCOALĂ GIMNAZIALA','SC.GEN.',
           'ȘC. GIMN.','ȘC.GIMN.','ȘCOAL GIMNAZIALĂ','ȘCOALA GIIMNAZIALĂ','ȘCOALA GIMNAIALĂ',
           'ŞCOALA GIMN.','SC. GEN.','SCOALA GEN.','SOALA GIMNAZIALA','ŞCOALA GIMNATZIALĂ','ŞC. GEN.','ŞC.GEN.',
           'ŞC. GIMN.','SCOALA GIMAZIALĂ','ŞCOALA CU CLASELE I- VIII','ŞCOALA GMNAZIALĂ','ŞCOALA GIMAZIALĂ',
           'ȘCOALA GIMNAZ.','SCOALA CU CL.I-VIII','SCOALA CU CL.I - VIII','SCOALA NORMALA','ȘCOALA GIM,','ȘCOALA GIM.',
           'FELSŐ NÉPISKOLA','SC. GIM.','SCOALA GENERELA','SC.GIM.','ŞC.GIM.','ŞCOAL GIMNAZIALĂ',
           'ȘCOALA GIMAZIALA','ȘCOALĂ GIMNAZIALĂ','ŞC.GIMN.','SC CU CLS. I-VIII','SCOALA CU CLS. I-VIII',
           'ŞCOALA CU CL. I-VIII.','ȘC. GIMNAZIALĂ','ȘC. GIM.','ŞCOALA CU CL -VIII','ŞC.CU CL.I-VIII','ŞCOALA I-VIII',
           'SC.GIMN','ȘC G.','ȘC GIMN.','SX.GIMN.','ȘC. G . ','ŞC. GIMNAZIALA','ŞC. I-VIII','ŞC.GIMNAZIALA','SCOALAGIMNAZIALA',
           'SCOALA CU CL. I-VIII','SCOALA CU CLASELE I -VIII','SCOALA CU CLASELE I VIII','SCOALA CU CLASELE I-VII',
           'SCOALA CU CLASELEI-VIII','SCOALA GIIMNAZIALA','ŞCOALA GIMANZIALĂ','SCOALA GIM CU PREDARE IN LIMBA MAGHIARA',
            'SCOALA GENERALA CU CL.I-VIII.','SCOALA GENERALA CLASELE I-VIII',
            'SC. CLS. I-VIII','SC. GIMN','SC GEN','SC GIMN','SC. GIMN','SCOALA GIMN','ŞCOALA GIMN','ȘCOALA GIMN','SC GIM',
            'SC. GIMNAZIAL'],
    'SPRM':['SCOALA PRIMARA','ŞCOALA CU CL. I-IV','SCOALA PRIMARĂ','ŞCOALA PRIMARĂ','ȘCOALA PRIMARĂ','SO4','SC. PRIM.','SC. PRIMARA','SCOALA CLS. I-IV',
           'ŞC.PRIMARĂ','ŞCOALA CU CLD. I-IV','ŞCOALA CU CLS.  I-IV.','ŞCOALA CU CLS. I-IV','SCOALA CU CLASELE I-IV',
           'ŞCOALA CU CL. I-IV','ŞCOALĂ PRIMARĂ','SC.PRIMARA.','ŞCOALA CU CLASELE I-IV','ŞCOALA PRIMARA','SC.PRIMARA',
           'SCOALA CU CL.I-IV.','SCOALA CU CL.I-IV','SCOALA PROMARA','CLASE PRIMARE','ALSÓ NÉPISKOLA','ȘCOALĂ PRIMARĂ','SC.PRIM.',
           'SCOALA PRIMARE','SCOALA PRIMARIA','SCOLA PRIMARA','SCOAL PRIMARA','SCOALA CL.I-IV','SCOALA CU CLS I-IV',
           'ST-ȘCOALA PRIMARA','ȘCOALA PRIMARA','ŞC.PRIM.','ŞC.PRIMARA','ŞCOALAPRIMARA','ŞCOASLA PRIMARĂ','ȘC. PRIMARĂ',
           'ŞCOALA PRIMAR','ŞCOLA PRIMAR','ȘCOALA PRIMARLĂ','SC.CL.I-IV','SC.CL I-IV','SC.CL. I-IV','SC.CLS.I-IV',
           'SC. CL.I-IV','SC. I-IV','SC. PRIMARĂ','SC.I-IV','SC.PEIMARA','SC.LC.I-IV','ȘCOLALA PRIMARĂ','ŞC.CU CL. I-IV',
           'SCOALAPRIMARA','SVÂC.PRIMARA','ŞC. PRIMARĂ','ŞCOALA PRIMATĂ','S04','SCOALA CU CL. I-IV',
           'SCOALA CU CLASELE I - IV','SCOALA CU CLS. I-IV','ŞCOALA CU CLS.I-IV','ȘCOALA CU CLS.I-IV','SC. CLS. I-IV',
           'SCOALA GENERALA CU CL.I-IV.','SCOALA GENERALA CU CL.I-IVI.','SCOALA GENERALA I-IV',
            'SC PRIM','SC.PR','SCOALA PRIM',],
    'SPSTL':['SCOALA POSTLICEALA','SCOALA SANITARA POSTLICEALA','ŞCOALA POSTLICEALĂ','ȘCOALA POSTLICEALĂ',
             'ŞCOALA SANITARĂ POSTLICEALĂ',
            'ȘCOALA POSTLICEALA','ȘCOALA SANITARĂ POSTLICEALĂ','ȘCOALA TEHNICĂ POSTLICEALĂ',
            'SCOALA POSTLICEALĂ'],
    'LICTEH':['COLEGIUL ECONOMIC','COLEGIUL TEHNIC','SCOALA PROFESIONALA','LIC. TEH.AGRICOL','LICEUL TEHN OLOGIC',
             'COLEGIUL TEHNOLOGIC','ŞCOALA PROFESIONALĂ','COLEGIUUL TEHNIC','LIC.TEHN.','COL.EC,,','LICEUL VOCATIONAL',
             'ȘCOALA PROFESIONALĂ','COLEGIUL AGRICOL','LIC. TEH.AGRICOL','LICEUL TEHN.AGR.','LICEU TEHNOLOGIC','COL.EC.','COL.EC .','COL. EC.',
             'COL.TEHN.DE','L.VOC.','LIC.TEH.','LIC.TEHNOL.','LICE. TEHNOLOGIC','LICEUL TEHNOLOGIC AGRICOL','LIC. TEHNOLOGIC','LICEUL TEHNOLOGIC',
             'LICEUL TEHN.','LICEUI TEHN.','LIC.TEHNOLOGIC.','LICEUL TEHOLOGIC','LIC. TEHN','LIC TEHN','C.T.'],
    'LISPRT':['LICEUL CU PROGRAM SPORTIV','CLUBUL SPORTIV SCOLAR','CLUB SPORTIV SCOLAR','CLUBUL SPORTIV ȘCOLAR',
             'CLUBUL ȘCOLAR SPORTIV','LICEUL VOCATIONAL CU PROGRAM SPORTIV','CLUBUL SPORTIV ŞCOLAR',
             'LICEUL SPORTIV','LPS ','L.P.S'],
    'CLBCOP':['CLUBUL COPIILOR','PALATUL COPIILOR','LUMEA COPIILOR','CLUBUL ELEVILOR'],
    'LITEOR':['LICEUL NAŢIONAL','LIC.TEO.','LIC. TEOR.','LIC.TEOR.','LICEUL TEORTIC','LIC. TEORETIC',
              'LICEUL TEORETIC','LIC T.','LIC.TEORETIC .','LIC.TEORETIC'],
    'LIPDGA':['LICEUL PEDAGOGIC','LICEUL VOCATIONAL PEDAGOGIC','LICEUL VOCAȚIONAL PEDAGOGIC','LICEUL VOCAŢIONAL PEDAGOGIC'],
    'LITOLX':['SEMINARUL TEOLOGIC','LICEUL TEOLOGIC ROM CAT.','LIC. TEOL.','LIC.ROM.CAT.','LIC.TEOL.ROM.',
             'LICEUL ROMANO CATOLIC','LICEUL TELOGIC ROMANO CATOLIC','LICEUL TEOL.ROM.CAT.','LICEUL ORTODOX',
             'LICEUL REFORMAT','LICEUL TEOLOGIC "BAPTIST"','LICEUL TEOLOGIC BAPTIST','LICEUL TEOLOGIC ROMANO CATOLIC',
             'LICEUL UNITARIAN','COLEGIUL REFORMAT','LICEUL TEOLOGIC ROMANO-CATOLIC','LICEUL ROM CAT',
             'LIC.VOCATIONAL REFORMAT','LICEUL VOCAȚIONAL REFORMAT','LICEUL VOCAŢIONAL REFORMAT','LICEUL VOCATIONAL REFORMAT'
             'LICEUL TEOLOGIC GRECO-CATOLIC','LICEUL TEOLOGIC'],
    'LIARTE':['COLEGIUL DE MUZICĂ','LICEUL DE MUZICA','LICEUL DE MUZICĂ','LICEUL DE ARTA','COLEGIUL DE ARTE',
             'LICEUL DE ARTE - PROGRAM SUPLIMENTAR DE MUZICA','LICEUL DE ARTE','LICEUL VOCAȚIONAL DE ARTĂ',
             'LICEUL DE ARTĂ','LIC. VOC.DE ARTA','LIC. DE ARTA','LIC.VOCATIONAL DE ARTA','LICEUL VOCATIONAL DE ARTA',
             'SCOALA DE ARTE SI MESERII'],
    'COLGNAT':['COLEGIUL NAŢIONAL','ŞCOALA NAŢIONALĂ','COL.NAŢ.','COL. NAT.','COLEGIUL NATIONAL',
              'COLEGIUL NAȚIONAL','COL. NAȚ.','COLEGIUL NAT.','COLEGIULNAŢIONAL','COLEGIUL NAŢIOANL','COL.NAT.',
              'COL. NAŢ.','SCOALA NATIONALA','ŞC MILITARĂ','COLEGIU NATIONAL','COL NAT','C.N.'],
    'GRDNRM':['GRADINITA CU PROGRAM NORMAL','GRĂDINITA CU PROGRAM NORMAL','GRĂDINIŢA CU PROGRAM NORMAL','GRĂDINIŢA CU PROGRAM SĂPTĂMÂNAL',
             'GRADINITA CU PROGRAM SAPTAMANAL','GRĂDINIȚA CU PROGRAM SAPTAMANAL','GRĂDINIȚA CU PROGRAM SĂPTĂMÂNAL',
             'GRĂDINIŢĂ CU PROGRAM NORMAL','GRĂDINIȚA CU PROGRAM NORMAL','GRAD.CU PROGRAM NORMAL','GRĂDINŢA CU PROGRAM NORMAL',
             'GRĂNIȚA CU PROGRAM NORMAL','GRADINITA CU PN','GRĂDINIŢA PN','GRĂDINIŢA PN','GRĂDINIŢĂ PN','G.P.N.',
             'GRADINIȚA CU PROGRAM NORMAL','GRĂDINIȚĂ CU PROGRAM NORMAL','GRAD CU PN','GRADINIŢA PN','GRĂD. CU PROGR. NORMAL',
             'GRADINITA CU P.N.','GRĂDINIŢA CU P.N.','GRĂDINIŢA CU PN','GRADINITA CU PROGR. NORMAL',
             'GRADINITA CU PROGRAMNORMAL','GRADINITA CU PROGRM NORMAL','GRĂDINIȚA CU PROPGRAM NORMAL','GRADINITA CU P.N',
             'GRĂDINIŢA CU PROG.NORMAL','GRĂDINIŢA CU PROGRAM NORNAL','GRADINITA CU PROGAM NORMAL','GRAD.PROG.NORMAL',
              'GRĂDINIŢA P.N.',
             'GRĂDINIŢĂ CU PROGRAM PRELUNGIT','GRĂDINIȚA CU PROGRAN NORMAL','GRADINITA CU PROGRASM NORMAL','GR.P.N.','GRAD P.N.',
             'GRAD. CU PROGRAM NORMAL','GRD CU PROGRAM NORMAL','GGĂDINIȚA CU PROGRAM NORMAL','GRADINTA CU PN','GRĂDINŢIA CU PROGRAM NORMAL',
             'GRADINITA DE COPII CU PROGRAM NORMAL','GRADINITA CU PRORGAM NORMAL','GRĂD.CU PROGR.NORM','GRADINITA DE COPII CU PN',
             'GRADINIŢA CU PROGRAM NORMAL','GRĂDCU PROGR.NORM','GRAD. CU PROG. NORMAL','GRĂDINIȚA PN','GRADINITA PN','GRAD.P.N.',
             'GRĂDINIȚA CU PR. NORMAL','GRADINITA DE COPII CU PRORAM NORMAL','G. P.N.','GRAD PN','GRĂD PN','GAD.P.N.','GRAD CU P.N.'
             'GRAD CU PR NORMAL','GRAD CU P.N.','GRAD CU PR. NORMAL','GRAD. CU P.N.','GRAD. P.N.','GRĂDINIȚA CU PN','GRAD. PN.',
             'GRAD. CU PR. NORMAL','GRAD.P. N.','GRAD.P.N.','GRAD.P.N','GRD.P.N.','GRAD.PN','GRADINITA P.N.','GRADINITA P.N',
             'GRADINITA PG.NORMAL.','GRADINITA PG.NORMAL','GRADINIŢA CU PN','GRAD.CU P N.','GRĂD.CU P.N.','GRADINITA PROG. NORMAL',
             'GR.PN.','GRAD.CU P.N.','GRĂD CU PR NORMAL','G.P.N','GRADDINITA CU PROGRAM NORMAL','GRADINTA CU PROGRAM NORMAL',
             'GRADINITA CU PRGRAM NORMAL','GRADINITA CUPROGRAM NORMAL','GNP','GPN','GRADINITA CU PROGR.NORM'],
    'GRDPLG':['GRADINITA CU PROGRAM PRELUNGIT','GRĂDINIŢA CU PROGRAM PRELUNGIT','GRĂDINIŢA PARTICULARĂ CU PROGRAM PRELUNGIT',
             'GRĂDINIȚA CU PROGRAM PRELUNGIT','G.P.P.','GPP','GRĂDINȚA CU PROGRAM PRELUNGIT','GRAD. CU PROGR. PRELUNGIT',
             'GRĂDINIŢA PP','GRADINITA PP','GRADINIȚA CU PROGRAM PRELUNGIT','GRĂDINIȚÁ CU PROGRAM PRELUNGIT','GRĂDINIȚĂ CU PROGRAM PRELUNGIT',
             'GRĂD. CU PROGR. PRELUNGIT','GRADINITA CU PP','GRĂDINIŢA CU PP',"GR[DINI'A CU PROGRAM PRELUNGIT",'GRADINITA CU P.P.',
             'GRĂDINIŢA CU P.P.','GRĂDINIȚA CU P.P.','GRĂDINIȚA CU PP','GRADINITACU P.P.','GRAD.CU P.P.','GRĂDINIŢA CU P. P.',
              'GRĂDINIŢA P.P.',
             'GRĂDINIŢA CU P.P','GRĂDINITA CU PROGRAM PRELUNGIT','GRAD. CU PR PRELUNGIT','GRAD. CU PR. PREL.','GRĂD.PROGR. PRELUNGIT',
             'GRAD. CU PR. PRELUNGIT','GRAD. CU PROGRAM PRELUNGIT','GRĂDINIŢA CU PR.PR.','GRĂDINIŢA CU PROG.PREL.','GRADINITA DE COPII PR.PRELUNGIT',
             'GRAD. CU PROG. PRELUNGIT','GRAD. PP','GR.PP.','GRĂDINIȚA PP','GR P.P.','GAD.P.P.','GPP','GR.P.P','GRAD P.P.','GRAD CU P.P.','GRAD CU P.P',
             'GRAD. CU P.P.','GRAD. P. P.','GRAD. P.P.','GRAD. P.P','GRAD.P.P.','GRAD.P.P','GRADINITA CU PR.PR.','GRADINITA PG.PREL.',
             'GRĂD.CU P.P.','GRADINITA PROG. PREL.','GRĂD PP','GRĂD CU PR PREL','G.P.P','GRADINITA CU PROGR.PREL.','GRĂDINIȚA CU PROGRAMPRELUNGIT',
             'GRAD CU PR PRELUNGIT','GRĂDINITA CU PRGRAM PRELUNGIT','GRADINITA P.P','GR P.P'],
    'GRPSC':['GRUP SCOLAR','G.Ş.','GRUPUL SCOLAR','GRUPUL ŞCOLAR','GRUPUL ȘCOLAR',
            'GR.SC. AGROIND.','GR.SC. EC.ADM.','GRUP ŞCOLAR','GRUPUL SC.AGR.IND.ALIM.','GRUPUL SC.EC.ADM.','GRUP SC.',
            'GR. SC.','GR.SC.','GR,ŞC.','GR. ŞC.'],
    'INSPSCJ':['INSPECTORATUL ŞCOLAR JUDEŢEAN','INSPECTORATUL ȘCOLAR JUDEȚEAN','CASA CORPULUI DIDACTIC','CENTRUL JUDETEAN DE RESURSE',
              'CENTRUL JUDEŢEAN DE RESURSE','CENTRU JUDEŢEAN DE RESURSE','CENTRUL JUDEȚEAN DE RESURSE','CENTRUL JUDEȚEAN DE EXCELENȚĂ',
              'INSPECTORATUL SCOLAR JUDETEAN','INSPECTORATUL ȘCOLAR AL JUDEȚULUI','INSPECTORATUL SCOLAR AL JUDETULUI',
              'CENTRUL DE RESURSE ȘI ASISTENȚĂ EDUCAȚIONALĂ'],
    'SCSPC':['CENTRUL SCOLAR DE EDUCATIE INCLUZIVA','LICEUL SPECIAL','ŞCOALA SPECIALĂ','CENTRUL ŞCOLAR PENTRU EDUCAŢIE INCLUZIVĂ','C.S E.I','C.S.E.I.',
            'CENTRUL DE EDUCATIE INCLUZIVA','CENTRUL ȘCOLAR DE EDUCAȚIE INCLUZIVĂ','CSEI','CENTRUL ȘCOLAR DE EDUCAŢIE INCLUZIVĂ',
            'CENTRUL ȘCOLAR PENTRU EDUCAȚIE INCLUZIVĂ','SCOALA SPECIALA','SCOALA SPECIALĂ','ȘCOALA SPECIALĂ','CU NEVOI SPECIALE',
            'CLASE SPECIALE','ȘC. SPEC.','CENTRUL JUDEȚEAN DE ASISTENȚĂ','CENTRUL SCOLAR PENTRU EDUCATIE INCLUZIVA',
             'CENTRUL SCOLAR PENTRU EDUCAȚIE INCLUZIVĂ',
            'CENTRUL ŞCOLAR DE EDUCAŢIE INCLUZIVĂ'],
    'STRUCT':['(TIPUL S)','(TIPUL ST)','STRUCTURĂ A ŞCOLII','(ST)','(S)','/ST','/ ST','/S','- ST','-ST','-S','- S','S -','UT -','UT-','S-','/ S','ST/','/UT',' UT',
             ' ST S','ST/'],
    'PERSJ':['(TIPUL PJ)','( PJ)','P.J','/PJ','-PJ','(PJ)','PJ -','PJ-','- PJ','-PJ','/ PJ','(PJ )',' PJ','(PJ, S)'],
    '':['"','”','„',',,','„','”',' / ','*',',','MUNICIPIUL','COMUNA ','ORAS','ORASUL',' COM ','STRUCTURA',"'",'(TIPUL )','(TIPUL)',
       'CU PREDARE IN LIMBA MAGHIARA'],
    ' ':['COM.'],
}
to_clean2={
    'SGMZ#':['SCOALA GENERALĂ','SCOALA GENERALA','ŞCOALA GENERALĂ','ȘCOALA GENERALĂ','SCOALA GENERAL','GIMN.','SCOLA GIMNAZIALA','SC CU CLS.','GIM. '],
    'SPRM#':['CU CLASELE I-IV'],
    'LICTEH#':[],
    'LISPRT#':[],
    'LITEOR#':['LICEUL'],
    'COLGNAT#':['COLEGIUL','COLEG ','COLEG. '],
    'GRDNRM#':['GRADINITA DE COPIII','GRĂDINIŢA DE COPII','GRADINITA DE COPII','GRADINITA','GRĂDINITA','GRĂDINIŢA','GRĂDINIŢA',
               'GRĂDINIŢĂ','GRĂDINIȚA','GRDINIȚA','GRADINIŢA','GRĂDĂNIŢA','GR.','GRADINIȚA','GRD.',
              'GRĂDINIȚĂ','GRIDINIŢA','GRĂD.','GRĂDINŢA','GĂDINIŢA','GRAD ','GRĂD ','GRAD.P.V.','GRAD.'],
    'GRDPLG#':[],
    'GRPSC#':[],    
    '':['STRUCT','PERSJ']    
}
to_clean_after={
    'GROF MAJLATH GUSZTAV KAROLY':['GMGK'],
    'FRANCISC NEUMAN':['FRATII NEUMAN'],
    'ADY ENDRE':['ADY ENDRE SERAL-MAGHIARA','ADY ENDRE TEORETIC','ADY ENDRE PJ'],
    'LICEUL HENRI COANDA':['HENRY COAND BECLEAN'],
    'NAPSUGAR':['NAPSUGAR ODORHEIU SECUIESC'],
    'CURCUBEU':['FUNDATIA CURCUBEUL','FUNDATIA CURCUBEU'],
    'CONSTANTIN BRANCUSI':['C-TIN BRANCUSI'],
    'TARNAVENI':['T-VENI'],
    'TIRGU MURES':['TG. MURES','TARGU MURES','TG.M.','TG.MURES','TÎRGU MUREȘ','TÎRGU MUREŞ'],
    'SANGEORGIU DE MURES':['SANG.DE MURES'],
    'SFANTU GHEORGHE':['SF.GHE.','SF.GHEORGHE','SF. GHEORGHE','SF.GH.'],
    'LUDUS':['LUDUȘ','LUDUŞ'],
    'DUMBRAVENI':['D-VENI'],
    
    #equivalencies
    'LICTEH':['GRPSC']
}
full_replace={
    'PJ':'PERSJ',
    'ST':'STRUCT',
    'STRUCTURA':'STRUCT',
    'S':'STRUCT',
    'UT':'STRUCT'
}
to_clean_k=16

Test outcomes

In [4316]:
convert_ro=False
dk=dg[dg['Megye']==m][['Név','telepules']].drop_duplicates().sort_values(by='Név') 
for d,i in enumerate(dk.T.iteritems()):
    nev=i[1]['Név'].upper()
    if convert_ro:
        nev=roman(nev)
    telepules=i[1]['telepules']
    clean_nev=cleaner(nev, telepules, convert_ro=convert_ro, test=True)
    to_print=True
    for s in list(to_clean.keys())[:to_clean_k]:
        if s in clean_nev: to_print=False
#     if '#' in clean_nev:
#         if 'GRDD' not in clean_nev:
#             to_print=True
#     to_print=True
    if nev in full_replace: to_print=False
    if to_print:
        print(d,clean_nev,'|',telepules,'|',nev)

In [4317]:
convert_ro=False
dv=gov[gov['Judet']==megye[m]][['Denumire','Localitate']].drop_duplicates().sort_values(by='Denumire')
for d,i in enumerate(dv.T.iteritems()):
    nev=i[1]['Denumire'].upper()
    if convert_ro:
        nev=roman(nev)
    telepules=i[1]['Localitate']
    telepules=roman(telepules)
    clean_nev=cleaner(nev, telepules, convert_ro=convert_ro, test=True)
    to_print=True
    for s in list(to_clean.keys())[:to_clean_k]:
        if s in clean_nev: to_print=False
#     if '#' in clean_nev:
#         if 'GRDNRM' in clean_nev:
#             to_print=True
#     to_print=True
    if nev in full_replace: to_print=False
    if to_print:
        print(d,clean_nev,'|',telepules,'|',nev) 

Do the matching

In [4318]:
gov_clean={}
for telepules in dv['Localitate'].unique():
    for g in dv[dv['Localitate']==telepules].T.iteritems():
        nev=g[1]['Denumire'].upper()
        nev=roman(nev)
        telepules=g[1]['Localitate']
        telepules=roman(telepules)
        clean_nev=cleaner(nev, telepules)
        if telepules not in gov_clean: gov_clean[telepules]={} 
        gov_clean[telepules][clean_nev]=g[0]

In [4319]:
clean_names=[]
guess_scores=[]
guessed_names=[]
for d,i in enumerate(dk.T.iteritems()):
    nev=i[1]['Név'].upper()
    nev=roman(nev)
    telepules=i[1]['telepules']
    clean_nev=cleaner(nev, telepules)
    clean_names.append(clean_nev)
    if telepules not in gov_clean:
        gov_clean[telepules]={clean_nev:'New location'}
    #first try token sort
    s=process.extractOne(clean_nev, list(gov_clean[telepules].keys()), scorer=fuzz.token_sort_ratio)
    if s[1]<70:
        #next try simple ratio
        s=s=process.extractOne(clean_nev, list(gov_clean[telepules].keys()), scorer=fuzz.ratio)
        if s[1]>84: #79 #84
            if clean_nev not in gov_clean[telepules]:
                gov_clean[telepules][clean_nev]='Inferred from location simple'
            guessed_names.append(s[0])
            guess_scores.append(s[1]+0.5)
        else:
            #if nothing found try all telepules in county
            s=process.extractOne(clean_nev, [item for sublist in \
                [list(gov_clean[t].keys()) for t in gov_clean] for item in sublist], scorer=fuzz.token_set_ratio)
            if s[1]>80: #80
                if clean_nev not in gov_clean[telepules]:
                    gov_clean[telepules][clean_nev]='Inferred from county token set'
                guessed_names.append(s[0])
                guess_scores.append(-s[1])
            else:
                #all telepules in county with simple
                s=process.extractOne(clean_nev, [item for sublist in \
                    [list(gov_clean[t].keys()) for t in gov_clean] for item in sublist], scorer=fuzz.ratio)
                if s[1]<85: #80 #85
                    if clean_nev not in gov_clean[telepules]:
                        gov_clean[telepules][clean_nev]='New school'
                    guessed_names.append(clean_nev)
                    guess_scores.append(0)
                else:
                    if clean_nev not in gov_clean[telepules]:
                        gov_clean[telepules][clean_nev]='Inferred from county simple'
                    guessed_names.append(s[0])
                    guess_scores.append(-s[1]-0.5)
    else:
        guessed_names.append(s[0])
        guess_scores.append(s[1])

dr=dk[['Név','telepules']]
dr['clean_names']=clean_names
dr['guessed_names']=guessed_names
dr['guess_scores']=guess_scores
mgov_clean[m]=gov_clean

In [4320]:
equivalencies={
    'Fehér': {},
    'Arad': {},
    'Bukarest': {},
    'Bákó': {},
    'Bihar': {},
    'Beszterce-Naszód': {},
    'Brassó': {},
    'Kolozs': {},
    'Kovászna': {},
    'Krassó-Szörény': {},
    'Hunyad': {},
    'Hargita': {},
    'Máramaros': {},
    'Maros': {},
    'Szeben': {},
    'Szatmár': {},
    'Szilágy': {},
    'Temes': {}
}

In [4321]:
dr['guessed_names2']=dr['guessed_names'].replace(equivalencies[m])
dr['guess_scores']=dr['guess_scores'].replace('YYY',0).replace('ZZZ',20)
dr=dr[['telepules','clean_names','Név','guessed_names2','guess_scores','guessed_names']]
dr.to_excel('data/clean/guessed/'+m+'.xlsx')

Run all counties, then continue:

In [4322]:
open('data/clean/mgov_clean.json','w').write(json.dumps(mgov_clean))

365395

...testing area...

In [4123]:
dr[((dr['guess_scores']<-80)&(-90<dr['guess_scores']))]

,telepules,clean_names,Név,guessed_names2,guess_scores,guessed_names


In [4112]:
dr[((dr['guess_scores']<20)&(-20<dr['guess_scores']))]

,telepules,clean_names,Név,guessed_names2,guess_scores,guessed_names
2078,TORMAC,GRDPLG TORMAC (ST S),"Grădiniţa P.P. Tormac (ST, S)",GRDPLG TORMAC (ST S),0,GRDPLG TORMAC (ST S)
4876,SANNICOLAU MARE,SGMZ NR.1SANNICOLAU MARE GRDPLG NR. 3SANNICOLA...,Şcoala Gimnazială Nr.1Sânnicolau MareGrădiniţa...,SGMZ NR.1SANNICOLAU MARE GRDPLG NR. 3SANNICOLA...,0,SGMZ NR.1SANNICOLAU MARE GRDPLG NR. 3SANNICOLA...


In [4041]:
dr[((dr['guess_scores']<80)&(70<dr['guess_scores']))]

,telepules,clean_names,Név,guessed_names2,guess_scores,guessed_names
1642,CAVNIC,LICTEH PINTEA VITEAZUL - POST ( PRESCOLAR ) HA...,"Col. Ec."" Pintea Viteazul""-S- Post ( prescolar...",LICTEH PINTEA VITEAZUL CAVNIC,72.0,LICTEH PINTEA VITEAZUL CAVNIC
4483,BAIA MARE,GRDNRM NR.25 BAIA MARE - GRUPA WALDORF,"GRADINITA CU PROGRAM PRELUNGIT NR.25, BAIA MAR...",GRDNRM NR. 25 BAIA MARE,76.0,GRDNRM NR. 25 BAIA MARE
7513,TARGU LAPUS,GRDNRM NR.2TG.LAPUS TARGU LAPUS,Grad.nr.2Tg.Lapus-S,GRDNRM TARGU LAPUS,73.0,GRDNRM TARGU LAPUS
4481,ULMENI,LICTEH INDUSTRIAL DR. FLORIAN ULMEANU ULMENI M...,"Grupul Scolar Industrial ""Dr. Florian Ulmeanu""...",LICTEH DR. FLORIAN ULMEANU ULMENI,79.0,LICTEH DR. FLORIAN ULMEANU ULMENI
2625,TARGU LAPUS,LICTEH TARGU LAPU,Grupul Şcolar Targu Lapus - PJ,SCSPC TARGU LAPUS,71.0,SCSPC TARGU LAPUS
7523,BAIA MARE,GRDNRM CU PROGRM PRELUNGIT ION CREANGA BAIA MARE,"Grădiniţa cu progrm prelungit ""Ion Creangă""-S",GRDNRM CU PROGRAM PLELUNGIT NR. 3 BAIA MARE,78.0,GRDNRM CU PROGRAM PLELUNGIT NR. 3 BAIA MARE
18768,ULMENI,LICTEH DR.F.ULMENU,LIC.TEHN.”DR.F.ULMENU”,LICTEH DR. FLORIAN ULMEANU ULMENI,72.0,LICTEH DR. FLORIAN ULMEANU ULMENI
10273,SEINI,LITEOR TEHN A GRDNRM ALEXIU BERINDE SEINI,LICEUL TEHN AGR.ALEXIU BERINDE (S),LICTEH ALEXIU BERINDE SEINI,74.0,LICTEH ALEXIU BERINDE SEINI
18769,SIGHETU MARMATIEI,LITEOR TEOERETIC SIGHETU MARMATIEI,LICEUL TEOERETIC,LISPRT SIGHETU MARMATIEI,72.0,LISPRT SIGHETU MARMATIEI
5434,SEINI,SGMZ ALEXIU BERINDE SEINI,"SCOALA GIMNAZIALA ""ALEXIU BERINDE"" SEINI - PJ",LICTEH ALEXIU BERINDE SEINI,77.0,LICTEH ALEXIU BERINDE SEINI


In [3674]:
loc='ZIMANDCUZ'
print(gov_clean[roman(loc)])
dv[dv['Localitate']==loc]

{'GRDNRM ZIMANDCUZ': 825, 'SPRM ZIMANDCUZ': 963}


,Denumire,Localitate
825,GRĂDINIŢA PN ZIMANDCUZ,ZIMANDCUZ
963,ŞCOALA PRIMARĂ ZIMANDCUZ,ZIMANDCUZ
